## Notebook steps 

1. Read stackholder reports on months when the extreme rainfall occured, reports are in xlsx file
2. Organize the CHRIPS data on that particular reported months
3. Plot the box plot of particular months
4. Plot the daily maps of particular months (excercise)
5. Check on the different masking methods and why there is need to choose based on use cases  


### Discussion points

1. Automate boring stuffs

2. Explore, test, develop in Jupyter Notebook

3. Production/ actual task in python script 

4. Script formatting in IDE

5. Version control the notebook and Python script

6. Unit testing practices 

In [ ]:
## read xlsx file for know the reported extreme rainfall months

In [ ]:
import pandas as pd
kmj_db=pd.read_excel('/srv/repo/IBF_workshop_data/kmj_daily_rainfall_chrips/kmj_impact_report.xlsx', sheet_name='Sheet1')
kmj_db

## get ncfiles 

In [ ]:
import glob


ncfiles_list=glob.glob('/srv/repo/IBF_workshop_data/kmj_daily_rainfall_chrips/*.nc')

ncfiles_list

In [ ]:
import xarray as xr

ds=xr.open_dataset(ncfiles_list[0])

ds

Note, the netcdf files are from this location, the download can be made for the years in xlsx fileshttps://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p05/chirps-v2.0.2007.days_p05.nc


## let us for loop and make the district wise csv file

In [ ]:
import pandas as pd

kmj_db=pd.read_excel('/srv/repo/IBF_workshop_data/kmj_daily_rainfall_chrips/kmj_impact_report.xlsx', sheet_name='Sheet1')

kmj_db['date1']=pd.to_datetime(kmj_db['date'],format='%Y%m')
kmj_db['year']=kmj_db['date1'].dt.year
kmj_db1=kmj_db.drop_duplicates('year')

year_list=kmj_db1['year'].tolist()


geodf = gp.read_file(f'/srv/repo/IBF_workshop_data/gis_files/kmj/Karamoja_9_districts.shp')
geodf.set_crs(epsg=4326,inplace=True)

output_folder='/srv/repo/IBF_workshop_data/kmj_daily_rainfall_chrips/raw_dist/'

for year in year_list:
    if year==2007:
        pass
    else:
        db_year=kmj_db[kmj_db['year']==year]
        xds = rioxarray.open_rasterio(f'{ncfile_path}chirps-v2.0.{year}.days_p05.nc')
        xds.rio.write_crs("epsg:4326",inplace=True)
        for idx, row in db_year.iterrows():
            dist_name=row['district']
            geodf1=geodf[geodf['admin2Name']==dist_name.title()]
            clipped = xds.rio.clip([geodf1.geometry.values[0]], geodf.crs)
            cds=clipped.to_dataset()
            month=int(datetime.strptime(str(row['date']), '%Y%m').date().strftime('%-m'))
            cds1=cds.loc[{"time":cds.time.dt.month==month}]
            cds1n=cds1.where((cds1>-9999), drop=True)
            cdb=cds1n.to_dataframe()
            cdb1=cdb.reset_index()
            cdb1.to_csv(f'{output_folder}{dist_name.lower()}_{year}_{month}.csv')

# plotting of chrips data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pylab


kmj_db=pd.read_excel('/srv/repo/IBF_workshop_data/kmj_daily_rainfall_chrips/kmj_impact_report.xlsx', sheet_name='Sheet1')

kmj_db['date1']=pd.to_datetime(kmj_db['date'],format='%Y%m')
kmj_db['year']=kmj_db['date1'].dt.year
#kmj_db1=kmj_db.drop_duplicates('year')

year_list=kmj_db1['year'].tolist()

kmj_db['test']=kmj_db['district']+kmj_db['date'].astype(str)
kmj_db1=kmj_db.drop_duplicates('test')

input_folder='/srv/repo/IBF_workshop_data/kmj_daily_rainfall_chrips/raw_dist/'

output_folder='/srv/repo/IBF_workshop_data/kmj_daily_rainfall_chrips/plot_dist/'
for idx, row in kmj_db[56:].iterrows():
    year=row['year']
    dist_name=row['district']
    month=int(datetime.strptime(str(row['date']), '%Y%m').date().strftime('%-m'))
    db=pd.read_csv(f'{input_folder}{dist_name.lower()}_{year}_{month}.csv')
    #db.boxplot(column='price', by='location')
    #fig = plt.figure()
    #fig.suptitle('test title', fontsize=20)
    db['Date']= pd.to_datetime(db['time'])
    db['Date']=db["Date"].dt.strftime('%Y-%m-%d')
    axes = db.boxplot(column='precip', by='Date',rot=90, fontsize=15,figsize=(16, 11))
    title_boxplot = dist_name
    plt.title( title_boxplot )
    plt.suptitle('') # that's what you're after
    #plt.show()
    pylab.savefig(f'{output_folder}{dist_name.lower()}_{year}_{month}.png')

## subset for each district
### test district subset 

In [ ]:
import rioxarray
import geopandas
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

geodf = geopandas.read_file('/srv/repo/IBF_workshop_data/gis_files/kmj/Karamoja_9_districts.shp')
xds = rioxarray.open_rasterio('/srv/repo/IBF_workshop_data/kmj_daily_rainfall_chrips/chirps-v2.0.2010.days_p05.nc')

geodf.set_crs(epsg=4326,inplace=True)

xds.rio.write_crs("epsg:4326",inplace=True)

# for idx, row in geodf.iterrows():
#     clipped = xds.rio.clip(geodf.geometry.values, geodf.crs)
#     dist_name=row['admin2Name']
#     local_path_nc=f'/home/chrips/dist_clips/{dist_name.lower()}.nc'
#     cds=clipped.to_dataset()
#     cds.to_netcdf(local_path_nc)
#     clipped=[]
fig=plt.figure(figsize=(10,10))

ax1=fig.add_subplot(331,projection=ccrs.PlateCarree())
clipped = xds.rio.clip([geodf.geometry.values[0]], geodf.crs)
cds=clipped.to_dataset()
cds1=cds.sel(time=slice('2010-01-01'))
cds1['precip'].sel(time=cds1['time'][0]).plot(ax=ax1,add_colorbar=False)
#legend()
#plt.title(geodf['admin2Name'].values[0])
ax1.legend().set_visible(False)
plotname=geodf['admin2Name'].values[0]
x=cds.sizes['x']
y=cds.sizes['y']
#plt.legend('',frameon=False)

plt.title(f'{plotname} with pixels {x} X {y}')
#legend()
#
ax2=fig.add_subplot(332,projection=ccrs.PlateCarree())
clipped = xds.rio.clip([geodf.geometry.values[1]], geodf.crs)
cds=clipped.to_dataset()
cds['precip'].sel(time=cds['time'][0]).plot(add_colorbar=False,ax=ax2)
plotname=geodf['admin2Name'].values[1]
x=cds.sizes['x']
y=cds.sizes['y']
plt.title(f'{plotname} with pixels {x} X {y}')
#
ax3=fig.add_subplot(333,projection=ccrs.PlateCarree())
clipped = xds.rio.clip([geodf.geometry.values[2]], geodf.crs)
cds=clipped.to_dataset()
cds['precip'].sel(time=cds['time'][0]).plot(add_colorbar=False,ax=ax3)
plotname=geodf['admin2Name'].values[2]
x=cds.sizes['x']
y=cds.sizes['y']
plt.title(f'{plotname} with pixels {x} X {y}')
#
ax4=fig.add_subplot(334,projection=ccrs.PlateCarree())
clipped = xds.rio.clip([geodf.geometry.values[3]], geodf.crs)
cds=clipped.to_dataset()
cds['precip'].sel(time=cds['time'][0]).plot(add_colorbar=False,ax=ax4)
plotname=geodf['admin2Name'].values[3]
x=cds.sizes['x']
y=cds.sizes['y']
plt.title(f'{plotname} with pixels {x} X {y}')
#
ax5=fig.add_subplot(335,projection=ccrs.PlateCarree())
clipped = xds.rio.clip([geodf.geometry.values[4]], geodf.crs)
cds=clipped.to_dataset()
cds['precip'].sel(time=cds['time'][0]).plot(add_colorbar=False,ax=ax5)
plotname=geodf['admin2Name'].values[4]
x=cds.sizes['x']
y=cds.sizes['y']
plt.title(f'{plotname} with pixels {x} X {y}')
#
ax6=fig.add_subplot(336,projection=ccrs.PlateCarree())
clipped = xds.rio.clip([geodf.geometry.values[5]], geodf.crs)
cds=clipped.to_dataset()
cds['precip'].sel(time=cds['time'][0]).plot(add_colorbar=False,ax=ax6)
plotname=geodf['admin2Name'].values[5]
x=cds.sizes['x']
y=cds.sizes['y']
plt.title(f'{plotname} with pixels {x} X {y}')
#
ax7=fig.add_subplot(337,projection=ccrs.PlateCarree())
clipped = xds.rio.clip([geodf.geometry.values[6]], geodf.crs)
cds=clipped.to_dataset()
cds['precip'].sel(time=cds['time'][0]).plot(add_colorbar=False,ax=ax7)
plotname=geodf['admin2Name'].values[6]
x=cds.sizes['x']
y=cds.sizes['y']
plt.title(f'{plotname} with pixels {x} X {y}')
#
ax8=fig.add_subplot(338,projection=ccrs.PlateCarree())
clipped = xds.rio.clip([geodf.geometry.values[7]], geodf.crs)
cds=clipped.to_dataset()
cds['precip'].sel(time=cds['time'][0]).plot(add_colorbar=False,ax=ax8)
plotname=geodf['admin2Name'].values[7]
x=cds.sizes['x']
y=cds.sizes['y']
plt.title(f'{plotname} with pixels {x} X {y}')
#
ax9=fig.add_subplot(339,projection=ccrs.PlateCarree())
clipped = xds.rio.clip([geodf.geometry.values[8]], geodf.crs)
cds=clipped.to_dataset()
cds['precip'].sel(time=cds['time'][0]).plot(add_colorbar=False,ax=ax9)
plotname=geodf['admin2Name'].values[8]
x=cds.sizes['x']
y=cds.sizes['y']
plt.title(f'{plotname} with pixels {x} X {y}')
